# Cat vs. Dog Classification
### Colin Santos and Cynthia Lai
---

Loading in Train and Test Images
-----

In [30]:
import os
import numpy as np
from tqdm import tqdm 
from PIL import Image
import tensorflow as tf
import keras
from imagenet_utils import decode_predictions
from imagenet_utils import preprocess_input
import random

In [2]:
def get_train_label(file_name):
    """Gives binary based on image name"""
    category = file_name.split('.')[0]
    if   category == 'cat': return 0 # [1, 0]
    elif category == 'dog': return 1 # [0, 1]

In [6]:
train_directory = 'train_small/'
# Form list of training images names
train_dir_files  = os.listdir(train_directory)
# Remove hidden MAC files
train_dir_files     = [i for i in train_dir_files if i!= '.DS_Store' ]
# Convert to 224 x 224 images
# !!! Modify image augmentation/normalization as necessary !!!
train_im_list = [Image.open(train_directory + im).resize((224,224)) for im in train_dir_files]

In [7]:
numPics = len(train_dir_files)

In [8]:
# Get training labels 
train_labels = [get_train_label(file) for file in train_dir_files]

In [9]:
#import _pickle as cPickle
import pickle as cPickle
f = open("im_list.pl", 'wb') # .pl is a pickle file
cPickle.dump(train_im_list, f)
f.close()

In [10]:
# Formatted data as (numpy arrays of) list of tuples of modified image and respective label or number
train_data = [(np.array(train_im, dtype=np.float64), np.array(train_label)) for (train_im, train_label) in zip(train_im_list, train_labels)]
#test_data  = [(np.array(test_im), np.array(test_number)) for (test_im, test_number) in zip(test_im_list, test_numbers)]

In [43]:
# just the data itself in np.array format
train_dat = [np.array(train_im, dtype=np.float64) for train_im in train_im_list]

array(0)

In [12]:
## IGNORE THIS

from keras.preprocessing import image as image_utils

im = image_utils.load_img(train_directory + train_dir_files[0], target_size=(224, 224))
test = image_utils.img_to_array(train_im_list[0])
test.shape

(224, 224, 3)

Convolutional Neural Network
----

#### First, work with VGG16 model.

In [13]:
from vgg16 import VGG16

## NOTE: 
1. Rerun line above train_data = [(np. ......]
2. Try to do the line below, and then remove the .astype(np.float64)

ISSUE IS THAT WE RUN INTO MEMORY ERROR

In [14]:
# train_data and test_data are 2 lists of np.array images
# need to preprocess images a bit more first

# make img file (1, 224, 224, 3)
temp = [np.expand_dims(image, axis = 0) for image in train_dat]
train_data2 = [preprocess_input(image, dim_ordering =  'tf') for image in temp]
#test_data = [preprocess_input(np.expand_dims(image, axis = 0)) for image in test_data]

In [15]:
# load model
model = VGG16(weights="imagenet")

vgg16.py:79: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu", name="block1_conv1")`
  x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv1')(img_input)
vgg16.py:80: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu", name="block1_conv2")`
  x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2')(x)
vgg16.py:84: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activation="relu", name="block2_conv1")`
  x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1')(x)
vgg16.py:85: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activation="relu", name="block2_conv2")`
  x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv2')(x)
vgg16.py:89: UserWar

K.image_dim_ordering: tf


In [16]:
# classify the image
preds = [model.predict(image) for image in tqdm(train_data2)]

100%|██████████| 1000/1000 [16:00<00:00,  1.07it/s]


### To speed up time, don't run the line above.
Instead, load the pickle file **preds.pl** and run the code 2 lines below

In [19]:
import pickle
f = open("preds.pl", 'wb') # .pl is a pickle file
pickle.dump(preds, f)
f.close()

In [ ]:
f = open("preds.pl", "rb")
preds = cPickle.load(f) # load PPMI matrix 

In [20]:
results = [decode_predictions(pred)[0] for pred in preds]
results[0:5]

[[u'n02123597', u'Siamese_cat'],
 [u'n03207941', u'dishwasher'],
 [u'n02123597', u'Siamese_cat'],
 [u'n02124075', u'Egyptian_cat'],
 [u'n02123159', u'tiger_cat']]

In [21]:
top_results_ids = [decode_predictions(pred)[0][0] for pred in preds]
top_results_lab = [decode_predictions(pred)[0][1] for pred in preds]
# labs = [res[1] for res in top_results]
# confs = [conf[2] for conf in top_results]

_Need opencv to use the below code._

In [23]:
# # display the predictions to our screen
# print("ImageNet ID: {}, Label: {}".format(inID, label))
# cv2.putText(orig, "Label: {}".format(label), (10, 30),
# 	cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
# cv2.imshow("Classification", orig)
# cv2.waitKey(0)

#### Next Steps:
1. Take **labels** and figure out if cat or dog based on classification output list. {:|:}
2. Calculate accuracy. {:|:}
3. Modify model if needed.
4. Try on other CNNs.
5. profit

In [24]:
# Load in Imagenet classications groupings for cats and dogs
with open('cats.txt') as f:
    cats = [line.rstrip()[0:9] for line in f]
with open('dogs.txt') as f:
    dogs = [line.rstrip()[0:9] for line in f]

In [25]:
# def cat_or_dog(class_id):
#     """Returns generalized classification of cat or dog"""
#     if class_id in cats: return 'cat'
#     if class_id in dogs: return 'dog'
#     return 'non' # non-binary case
    
def one_or_zero(class_id):
    """Returns generalize classification of cat or dog as bool"""
    if class_id in cats: return 0
    if class_id in dogs: return 1
    return 420 # non-binary case
    
from operator import eq
def calculate_accuracy(pred_labels, true_labels):
    """Calculates accuracy"""
    return sum(map(eq, pred_labels, true_labels)) / float(len(true_labels))

In [44]:
simple_class = [one_or_zero(id) for id in top_results_ids]
[(class_,lab_) for (class_,lab_) in zip(simple_class, top_results_lab)][0:5]

[(0, u'Siamese_cat'),
 (420, u'dishwasher'),
 (0, u'Siamese_cat'),
 (0, u'Egyptian_cat'),
 (0, u'tiger_cat')]

In [45]:
calculate_accuracy(simple_class, train_labels)

0.819